In [46]:
import string

from transformers import BertTokenizer, BertModel
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn
from torch.nn.functional import pad

import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [61]:
Train = pd.read_csv('train_spam.csv', names=['labels', 'text'], header=0)

In [48]:
Train.head()

,labels,text
0,ham,make sure alex knows his birthday is over in f...
1,ham,a resume for john lavorato thanks vince i will...
2,spam,plzz visit my website moviesgodml to get all m...
3,spam,urgent your mobile number has been awarded wit...
4,ham,overview of hr associates analyst project per ...


In [124]:
for i in Train.sample(10).text:
    print(i)

yeah i imagine he would be really gentle unlike the other docs who treat their patients like turkeys
yar i wanted 2 scold u yest but late already i where got zhong se qing you if u ask me b4 he ask me then ill go out w u all lor n u still can act so real
on mon 1635465 sep 1635465 russell turpin wrote adam beberg considering 1635465 of the fake job posting i see are for embedded systems or device drivers c still rules the world there is a lot of c in the embedded world with static object allocation and a few other programming techniques performance differences disappear but c gives a boost in development and maintainability agreed not much difference there with c it just doesnt seem as wrong to be crawling around in registers and things quite frankly you cant fit that big of a project into a 1635465k rom so large project issues dont matter as much in the embedded world and in the realtime space or when you have data coming in at 1635465gbit sec fibrechannel every cycle does count the r

Как можно понять (несколько раз посемплил), все текста очищены от знаков пунктуации и переведены к нижнемы регистру, осталось только привести лейблы к численному виду.

In [66]:
Train.labels = Train.labels.apply(lambda x: int(x == 'spam'))

## №1: Байесовский клаасификатор

In [81]:
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(Train.text)

X_train, X_valid, y_train, y_valid = train_test_split(X, Train.labels,
                                                      test_size=0.2)

nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

pred = nb_model.predict(X_valid)

roc_auc_score(y_valid, pred)

0.9084384019444023

Наивный Байес показал отличные показатели на тестовой выборке (здесь и дальше называю valid чтобы не путать с итоговой тестовой)

## №2: Эмбеддинги BERT

Здесь я решил использовать скрытые представления BERT как эмбединги предложений и прогонять их через полносвязную сетку

In [56]:
Train.text[5569]

'요즘 뜨는 직종 best 1635465 금융 it 방송 뷰티 인테리어 1635465대 젊은이들이 뽑은 일하고 싶은 분야 로 금융 it 방송 뷰티 패션 인테리어 건축분야가 선정되었다 이들 중에서도 금융분야는 급여가 상대적으로 높고 주1635465일제 근무에 앞서있는 외국계 은행 이 it분야는 게임 분야가 방송분야는 요즘 신세대 대학생들이 가장 선호하는 직업인 홈쇼핑의 쇼핑호스트 와 케이블방송의 vj 를 뷰티 패션분야는 美에 대한 관심고조로 인해 피부관리사 또는 스타일리스트 인테리어 건축분야에서는 리모델링 붐에 힘입어 인테리어 쪽이 높은 순위로 꼽혔다 그럼 이처럼 높은 주가를 구가하고 있는 직종에 근무하기 위해서는 어떤 자격요소를 갖추는 것이 좋을까 본격적인 여름이 시작되는 이때 뜨는 직종으로의 확실한 준비를 해보자 금융 1635465년 말 imf가 시작될 무렵만 해도 은행 보험 증권계의 한파는 그 어느분야보다도 치열했고 무차별적인 구조조정으로 인해 많은 직장인들이 거리로 내몰리다시피 되었었다 하지만 1635465년 현재 카드사의 호황을 시작으로 은행권 증권 보험분야의 경력직 수요가 많아지고 있으며 신입직의 채용 또한 많아지고 있다 올 하반기까지 인력시장은 맑음 상태 금융분야는 자격증이 있으면 있을수록 좋다 금융권에서 유용하게 쓰이는 자격증으로는 투자상담사 금융자산관리사 fp 재무위험관리사 frm 등 특히 증권사에 근무할 이들이라면 투자상담사나 금융자산관리사의 자격증은 기본이 되고 있으니 반드시 취득해두도록 하자 hyperlink 시험일정 보기 it 올해들면서 it 인터넷 업계의 수익구조가 좋아지고 있으며 인터넷'

Эту запись я удаляю как "выброс", т.к. на ней падает BertModel

In [57]:
Train = Train.drop(5569).reset_index(drop=True)

In [ ]:
model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name).to(device)
bert_model.eval()

In [12]:
train, valid = train_test_split(Train, test_size=0.2)
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)

In [15]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe['text'][idx]
        label = self.dataframe['labels'][idx]

        inputs = self.tokenizer(
            text, max_length=self.max_length, padding='max_length',
            truncation=True, return_tensors='pt'
        )

        return inputs['input_ids'].flatten().to(device),  \
                inputs['attention_mask'].flatten().to(device), \
                torch.tensor(label, dtype=float).to(device)


class Classifier(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Classifier, self).__init__()
        self.flatten = nn.Flatten(start_dim=1)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size * 4)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size * 4, hidden_size * 4)
        self.relu = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size * 4, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        return self.sigmoid(x)


In [16]:
BATCH_SIZE = 128
MAX_LEN = 512

INPUT_SIZE = MAX_LEN * 768
HIDDEN_SIZE = 128

NUM_EPOCHS = 3

In [18]:
train_loader = DataLoader(CustomDataset(train, bert_tokenizer, MAX_LEN),
                          batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(CustomDataset(valid, bert_tokenizer, MAX_LEN),
                          batch_size=BATCH_SIZE, shuffle=True)


classifier_model = Classifier(INPUT_SIZE, HIDDEN_SIZE).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier_model.parameters(), lr=0.001)

Note: я понимаю, что будет быстрее получить сначала эмбеддинги предложений, и потом обучать сетку, но colab не вывозит такого и "CUDA out of memory"

In [20]:
for epoch in range(NUM_EPOCHS):
    print(f'epoch{epoch + 1}')

    classifier_model.train()
    losses = []

    for input_ids, attention_masks, labels in tqdm(train_loader):
        with torch.no_grad():
            embeddings = bert_model(
                input_ids, attention_mask=attention_masks
            ).last_hidden_state
        optimizer.zero_grad()
        pred = classifier_model(embeddings)
        loss = criterion(pred.flatten(), labels)
        loss.backward()
        losses.append(loss.item())
        optimizer.step()

    print(f'train loss {torch.mean(torch.tensor(losses))}')

    losses = []
    classifier_model.eval()

    with torch.no_grad():
        for input_ids, attention_masks, labels in tqdm(valid_loader):
            embeddings = bert_model(
                input_ids, attention_mask=attention_masks
            ).last_hidden_state
            pred = classifier_model(embeddings)
            loss = criterion(pred.flatten(), labels)
            losses.append(loss.item())

    print(f'valid loss {torch.mean(torch.tensor(losses))}')

epoch1


100%|██████████| 102/102 [08:21<00:00,  4.92s/it]


train loss 174.4746551513672


100%|██████████| 26/26 [02:03<00:00,  4.75s/it]


valid loss 163.73121643066406
epoch2


...

valid loss 174.63299560546875
epoch3


100%|██████████| 102/102 [08:18<00:00,  4.88s/it]


train loss 171.80836486816406


100%|██████████| 26/26 [02:04<00:00,  4.78s/it]

valid loss 167.3802032470703


In [21]:
real = []
scores = []


with torch.no_grad():
    for input_ids, attention_masks, labels in tqdm(valid_loader):
        embeddings = bert_model(
            input_ids, attention_mask=attention_masks
        ).last_hidden_state
        pred = classifier_model(embeddings)

        scores = scores + pred.flatten().tolist()
        real = real + labels.tolist()

print(roc_auc_score(real, scores))


100%|██████████| 26/26 [02:05<00:00,  4.82s/it]

0.7637025559112701


Результаты получились относительно неплохие, это решение можно явно улучшить (хотя бОльшим временем обучения или числом параметров сетки) но на бесплатой GPU от колаба это проблематично :)

## Результаты:
Лучщим решением оказался наивный бейес (0.91 против 0.76)

In [119]:
Test = pd.read_csv('test_spam.csv')

In [120]:
Test['score'] = nb_model.predict_proba(vectorizer.transform(Test.text))[:, 1]

In [123]:
Test.to_csv('results.csv', index=False)